In [32]:
import sys

sys.path.append("../")

import os
import numpy as np
import cv2
import pandas as pd

from pedrec.configs.dataset_configs import get_sim_dataset_cfg_default, PedRecTemporalDatasetConfig
from pedrec.configs.pedrec_net_config import PedRecNet50Config
from pedrec.models.constants.dataset_constants import DatasetType
from pedrec.visualizers.skeleton_visualizer import draw_skeleton
from pedrec.datasets.pedrec_temporal_dataset import PedRecTemporalDataset
from pedrec.models.constants.action_mappings import ACTION
from pedrec.models.constants.sample_method import SAMPLE_METHOD
from pedrec.configs.app_config import AppConfig
from random import randint

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
%matplotlib widget

In [8]:
dataset_dir = "data/datasets/Conti01/"
dataset_file = "rt_conti_01_val_FIN.pkl"
dataset_results_file = "C01F_pred_df_experiment_pedrec_p2d3d_c_o_h36m_sim_mebow_0_allframes.pkl"

In [21]:
cfg = PedRecNet50Config()
app_cfg = AppConfig()

dataset_cfg = PedRecTemporalDatasetConfig(
    flip=True,
    scale_factor=0.25,
    rotation_factor=0,
    skeleton_3d_range=3000,
    img_pattern="view_{cam_name}-frame_{id}.{type}",
    subsample=1,
    subsampling_strategy=SAMPLE_METHOD.SYSTEMATIC,
    gt_result_ratio=0,
    use_unit_skeleton=True,
    min_joint_score=0,
    add_2d=False
)
dataset = PedRecTemporalDataset(dataset_dir,
                                  dataset_file,
                                  DatasetType.VALIDATE,
                                  dataset_cfg,
                                  app_cfg.inference.action_list,
                                  None,
                                  pose_results_file=dataset_results_file)
dataset_length = len(dataset)


In [26]:
fig, ax = plt.subplots(3,3, figsize=(10,10))

count = 0
for i in range(0, 3):
    for j in range(0, 3):
        idx = randint(0, dataset_length)
        entry = dataset[idx]
        # entry = dataset[count + 606]
        ehpi_3d, action_id = entry
        actions = f"{idx}: "
        for idx, is_active in enumerate(action_id):
            if is_active == 1:
                actions = f"{actions}{app_cfg.inference.action_list[idx].name}, "
        img = cv2.cvtColor(ehpi_3d, cv2.COLOR_BGR2RGB)
        ax[i, j].imshow(img)
        ax[i, j].set_title(actions)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
gts = np.load("/home/dennis/code/latex/Dissertation/data/ehpi_c01_results/gts.npy")
preds = np.load("/home/dennis/code/latex/Dissertation/data/ehpi_c01_results/preds.npy")
print(gts.dtype)
print(preds.dtype)

float64
float64


In [28]:
labels = []
for action in app_cfg.inference.action_list:
  labels.append(action.name)

In [44]:
for i in range(len(labels)):
  a = confusion_matrix(gts[:, i], preds[:, i])
  disp = ConfusionMatrixDisplay(a)
  disp.plot()

TypeError: __init__() got an unexpected keyword argument 'title'